In [42]:
import os
from pathlib import Path
import openai
import pandas as pd
import logging
from IPython.display import clear_output
from copy import deepcopy
import networkx as nx
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

from utils.utils import log_completion_call, default_arguments_for_openai_classification, default_arguments_for_openai_generation, default_arguments_for_openai_validation, \
    ask_gpt, get_feedback, ask_gpt_get_feedback_and_log, create_openai_completion, get_label_for_correct_or_incorrect_completion, path_to_unused_finetuning_data_folder
from prompts import q2, kshot_prompts

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# validate answer how it works

In [ ]:
prompt = """
Convert the information in the first format into the second. Below are some correct examples:

Question: How does precision work?
Applicant's response: Precision is a model metric that measures what fraction of the actual positives the model predicted as positive.
Correct response: Precision is a model metric that measures what fraction of the predicted positives the model predicted correctly.
Validation of response: incorrect

Current section chat:
Jim: How does precision work?
Applicant: Precision is a model metric that measures what fraction of the actual positives the model predicted as positive.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Precision is a model metric that measures what fraction of the predicted positives the model predicted correctly.
Validation of response: 1


Question: How does precision work?
Applicant's response: Precision is a model metric
Correct response: Precision is a model metric that measures what fraction of the predicted positives the model predicted correctly.
Validation of response: incomplete

Current section chat:
Jim: How does precision work?
Applicant: Precision is a model metric
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Precision is a model metric that measures what fraction of the predicted positives the model predicted correctly.
Validation of response: 2


Question: How does forward selection work?
Applicant's response: With forward selection, you start with zero features and keep adding features till you have a model you like.
Correct response: With forward selection, you start with a model with zero features and add the feature which has the best score. The score is chosen beforehand, like the p value of a t test. We keep doing this iteratively until a stopping criterion is reached, like a max number of features or a min increase in performance by adding a feature. 
Validation of response: incomplete

Current section chat:
Jim: How does forward selection work?
Applicant: With forward selection, you start with zero features and keep adding features till you have a model you like.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: With forward selection, you start with a model with zero features and add the feature which has the best score. The score is chosen beforehand, like the p value of a t test. We keep doing this iteratively until a stopping criterion is reached, like a max number of features or a min increase in performance by adding a feature. 
Validation of response: 2


Question: How does one-hot encoding work?
Applicant's response: One-hot encoding converts each unique categorical value to a number.
Correct response: One-hot encoding converts each unique categorical value to a binary vector.
Validation of response: incorrect


Question: How does standard scaler work?
Applicant's response: it normalizes the data
Correct response: Standard scaler transforms the data by subtracting the mean and dividing by the standard deviation, so that the resulting distribution has a mean of 0 and a standard deviation of 1.
Validation of response: incomplete

Current section chat:
Jim: How does standard scaler work?
Applicant: it normalizes the data
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Standard scaler transforms the data by subtracting the mean and dividing by the standard deviation, so that the resulting distribution has a mean of 0 and a standard deviation of 1.
Validation of response: 2


Question: How does standard scaler work?
Applicant's response: it gives the data a mean of 0 and standard deviation of 1
Correct response: Standard scaler transforms the data by subtracting the mean and dividing by the standard deviation, so that the resulting distribution has a mean of 0 and a standard deviation of 1.
Validation of response: correct

Current section chat:
Jim: How does standard scaler work?
Applicant: it gives the data a mean of 0 and standard deviation of 1
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Standard scaler transforms the data by subtracting the mean and dividing by the standard deviation, so that the resulting distribution has a mean of 0 and a standard deviation of 1.
Validation of response: 3


Question: How does mean imputing work?
Applicant's response: it imputes the missing value with the last known value of that feature
Correct response: Mean imputing replaces missing values with the mean of the feature.
Validation of response: incorrect

Current section chat:
Jim: How does mean imputing work?
Applicant: it imputes the missing value with the last known value of that feature
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Mean imputing replaces missing values with the mean of the feature.
Validation of response: 1


Question: How does mean imputing work?
Applicant's response: it fills the missing values
Correct response: Mean imputing replaces missing values with the mean of the feature.
Validation of response: incomplete

Current section chat:
Jim: How does mean imputing work?
Applicant: it fills the missing values
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Mean imputing replaces missing values with the mean of the feature.
Validation of response: 2


Question: How does grid search work?
Applicant's response: creates a grid of parameters and picks the best hyperparameters
Correct response: Grid search is an algorithm tuning technique that exhaustively searches through a manually specified subset of the hyperparameter space of a learning algorithm for the best performing set of hyperparameters.
Validation of response: incomplete

Current section chat:
Jim: How does grid search work?
Applicant: creates a grid of parameters and picks the best hyperparameters
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Grid search is an algorithm tuning technique that exhaustively searches through a manually specified subset of the hyperparameter space of a learning algorithm for the best performing set of hyperparameters.
Validation of response: 2


Question: How does SMOTE work?
Applicant's response: SMOTE synthetically generates observations
Correct response: SMOTE (Synthetic Minority Oversampling Technique) is an oversampling technique that synthetically generates observations of the minority class by interpolating between existing observations.
Validation of response: incomplete

Current section chat:
Jim: How does SMOTE work?
Applicant: SMOTE synthetically generates observations
Details:
Question type: how it works
Is validation correct: 1
Correct Response: SMOTE (Synthetic Minority Oversampling Technique) is an oversampling technique that synthetically generates observations of the minority class by interpolating between existing observations.
Validation of response: 2


Question: How does Lasso work?
Applicant's response: It is a linear model that helps to select features for you.
Correct response: A lasso model is a type of regularization technique that helps to reduce model complexity and mitigate overfitting. It does this by introducing a penalty term to the cost function of the model, which penalizes the magnitude of the coefficients of the predictors in the model. This encourages the model to only include the most important predictors and reduces the complexity of the model. The result is a more accurate and reliable model.
Validation of response: incomplete

Current section chat:
Jim: How does Lasso work?
Applicant: It is a linear model that helps to select features for you.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: A lasso model is a type of regularization technique that helps to reduce model complexity and mitigate overfitting. It does this by introducing a penalty term to the cost function of the model, which penalizes the magnitude of the coefficients of the predictors in the model. This encourages the model to only include the most important predictors and reduces the complexity of the model. The result is a more accurate and reliable model.
Validation of response: 2


Question: How does one-hot encoding work?
Applicant's response: Sorry, i don't know.
Correct response: One-hot encoding converts each unique categorical value to a binary vector.
Validation of response: incorrect

Current section chat:
Jim: How does one-hot encoding work?
Applicant: Sorry, i don't know.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: One-hot encoding converts each unique categorical value to a binary vector.
Validation of response: 0


Question: In feature engineering, what does it mean when we create polynomial features?
Applicant's response: What do you mean?
Correct response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways. This can include taking the product of two features, or taking the square or cube of a single feature.
Validation of response: ADNU

Current section chat:
Jim: In feature engineering, what does it mean when we create polynomial features?
Applicant: What do you mean?
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways. This can include taking the product of two features, or taking the square or cube of a single feature.
Validation of response: 0


Question: In feature engineering, what does it mean when we create polynomial features?
Applicant's response: I don't understand.
Correct response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways. This can include taking the product of two features, or taking the square or cube of a single feature. This allows us to capture non-linear relationships between the features and the target variable.
Validation of response: ADNU

Current section chat:
Jim: In feature engineering, what does it mean when we create polynomial features?
Applicant: I don't understand.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways. This can include taking the product of two features, or taking the square or cube of a single feature. This allows us to capture non-linear relationships between the features and the target variable.
Validation of response: 0


Question: In feature engineering, what does it mean when we create polynomial features?
Applicant's response: We create new features using the existing features? I'm not really sure what you're asking.
Correct response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways, such as squaring, cubing, or taking the square root of the features.
Validation of response: ADNU

Current section chat:
Jim: In feature engineering, what does it mean when we create polynomial features?
Applicant: We create new features using the existing features? I'm not really sure what you're asking.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: Polynomial features are created by taking the existing features and creating new features by combining them in different ways, such as squaring, cubing, or taking the square root of the features.
Validation of response: 0


Question: calculate number of accidents prevented by using my model
Applicant's response: count the number of true positives - assuming every predicted positive was inspected to prevent an accident.
Correct response: To calculate the number of accidents prevented by using the model, you would need to count the number of true positives and multiply it by the estimated probability of an accident occurring if the predicted positive had not been inspected.
Validation of response: incomplete

Current section chat:
Jim: calculate number of accidents prevented by using my model
Applicant: count the number of true positives - assuming every predicted positive was inspected to prevent an accident.
Details:
Question type: how it works
Is validation correct: 1
Correct Response: To calculate the number of accidents prevented by using the model, you would need to count the number of true positives and multiply it by the estimated probability of an accident occurring if the predicted positive had not been inspected.
Validation of response: 2


Question: calculate total dollar cost of solution
Applicant's response: use model confusion matrix to calculate total dollars cost of the solution
Correct response: The total dollar cost of a solution can be calculated by multiplying the cost of a false positive and false negative by the number of false positives and false negatives in the model's confusion matrix.
Validation of explanation: incomplete

Current section chat:
Jim: calculate total dollar cost of solution
Applicant: use model confusion matrix to calculate total dollars cost of the solution
Details:
Question type: how it works
Is validation correct: 1
Correct Response: The total dollar cost of a solution can be calculated by multiplying the cost of a false positive and false negative by the number of false positives and false negatives in the model's confusion matrix.
Validation of response: 2


Question: calculate number of accidents prevented by using my model
Applicant's response: subtract the number of false negatives after using my model from the number of false negatives before using my model
Correct response: The number of accidents prevented by using a model can be calculated by subtracting the number of false negatives after using the model from the number of false negatives before using the model.
Validation of explanation: correct

Current section chat:
Jim: calculate number of accidents prevented by using my model
Applicant: subtract the number of false negatives after using my model from the number of false negatives before using my model
Details:
Question type: how it works
Is validation correct: 1
Correct Response: The number of accidents prevented by using a model can be calculated by subtracting the number of false negatives after using the model from the number of false negatives before using the model.
Validation of response: 3


Question: calculate number of accidents prevented after using my model
Applicant's response: calculate difference in the number of true positives between the client's approach before my solution and with my solution
Correct response: The number of accidents prevented after using a model can be calculated by subtracting the number of true positives from the model's confusion matrix before the model was implemented from the number of true positives after the model was implemented.
Validation of explanation: correct

Current section chat:
Jim: calculate number of accidents prevented after using my model
Applicant: calculate difference in the number of true positives between the client's approach before my solution and with my solution
Details:
Question type: how it works
Is validation correct: 1
Correct Response: The number of accidents prevented after using a model can be calculated by subtracting the number of true positives from the model's confusion matrix before the model was implemented from the number of true positives after the model was implemented.
Validation of response: 3


Question: calculate number of accidents prevented after using my model
Applicant's response: subtract the number of true positives before using my model from the number of true positives after using the model
Correct response: The number of accidents prevented after using a model can be calculated by subtracting the number of true positives from the model's confusion matrix before the model was implemented from the number of true positives after the model was implemented.
Validation of explanation: correct

Current section chat:
Jim: calculate number of accidents prevented after using my model
Applicant: subtract the number of true positives before using my model from the number of true positives after using the model
Details:
Question type: how it works
Is validation correct: 1
Correct Response: The number of accidents prevented after using a model can be calculated by subtracting the number of true positives from the model's confusion matrix before the model was implemented from the number of true positives after the model was implemented.
Validation of response: 3"""

# ask

In [2]:
previous_format = kshot_prompts.ask_applicant_questions_set1

In [28]:
prompt_to_convert_formats_template = """Convert the information in the first format into the second. Note that the newlines are part of the format. 

First format:
What Applicant used: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>

Second format:
Current section chat:


Details:
Approach: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>


Examples:
{0}
"""

In [12]:
previous_format_examples = previous_format.split("""Below are some examples:

""")[-1]

In [19]:
previous_format_examples_list = previous_format_examples.split("\n\n")[:-1]

In [20]:
previous_format_examples_list

['What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the difference between precision and recall?',
 'What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What are the advantages and disadvantages of using precision as a metric?',
 'What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the difference between precision and accuracy?',
 'What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What are the differences between precision, recall, and accuracy?',
 'What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the relationship between precision and recall?',
 'What Applicant used: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What are the assumptions behind the calculation of precision?',
 'What Applicant used: precision\nQuestion type: how it works\nIs quest

In [29]:
ask_gpt(prompt_to_convert_formats_template.format(previous_format_examples_list[0]))

Convert the information in the first format into the second. Note that the newlines are part of the format. 

First format:
What Applicant used: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>

Second format:
Current section chat:


Details:
Approach: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>


Examples:
What Applicant used: precision
Question type: how it works
Is question correct: 0
Jim: What is the difference between precision and recall?
Current section chat:

Details:
Approach: precision
Question type: how it works
Is question correct: 0
Jim: What is the difference between precision and rec

'Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the difference between precision and recall?'

In [30]:
new_format_examples_list = [ask_gpt(prompt_to_convert_formats_template.format(i)) for i in previous_format_examples_list]

Convert the information in the first format into the second. Note that the newlines are part of the format. 

First format:
What Applicant used: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>

Second format:
Current section chat:


Details:
Approach: $<the approach Applicant used to solve the problem>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
Jim: $<a question about Applicant's approach>


Examples:
What Applicant used: gridsearch
Question type: how it works
Is question correct: 1
Jim: Can you explain how Grid Search works?
Current section chat:
Jim: Can you explain how Grid Search works?

Details:
Approach: Grid Search
Question type: How it works
Is question correct: 1


In [31]:
new_format_examples_list

['Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the difference between precision and recall?',
 'Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What are the advantages and disadvantages of using precision as a metric?',
 'Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What is the difference between precision and accuracy?',
 'Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0\nJim: What are the differences between precision, recall, and accuracy?',
 'Current section chat:\nJim: What is the relationship between precision and recall?\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct: 0',
 'Current section chat:\n\nDetails:\nApproach: precision\nQuestion type: how it works\nIs question correct:

In [36]:
ask_applicant_questions = """Jim is interviewing Applicant for a job as a Data Scientist. Jim knows all concepts in Data Science and always applies them correctly.

Jim asks Applicant questions about certain subjects, in the following format:

Current section chat:
Jim: $<asks a question>
Applicant: $<response>
$<Jim and Applicant may have a longer conversation, or none at all>

Details:
Approach: $<the subject Jim is to ask a question about>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
$<intermediate step to help come up with the right question. Jim may have more chain of thought steps, or none at all>
Jim: $<jim's question about Applicant's approach>

Below are some examples:

"""

In [37]:
for i in new_format_examples_list:
    ask_applicant_questions += i + "\n\n\n"

In [38]:
print(ask_applicant_questions)

Jim is interviewing Applicant for a job as a Data Scientist. Jim knows all concepts in Data Science and always applies them correctly.

Jim asks Applicant questions about certain subjects, in the following format:

Current section chat:
Jim: $<asks a question>
Applicant: $<response>
$<Jim and Applicant may have a longer conversation, or none at all>

Details:
Approach: $<the subject Jim is to ask a question about>
Question type: $<what type of question jim should ask>
Is question correct: $<1 or 0 - whether the question jim asks is of the right type or not>
$<intermediate step to help come up with the right question. Jim may have more chain of thought steps, or none at all>
Jim: $<jim's question about Applicant's approach>

Below are some examples:

Current section chat:

Details:
Approach: precision
Question type: how it works
Is question correct: 0
Jim: What is the difference between precision and recall?


Current section chat:

Details:
Approach: precision
Question type: how it wor

In [43]:
ask_gpt(kshot_prompts.ask_applicant_questions_set1_v0.format("Current section chat:\nJim: I'd like to talk about feature engineering.\nDetails:\nApproach: Polynomial features\nQuestion type: How it works\nIs question correct: 1"))

'Jim: How do polynomial features work?'